In [442]:
import math
from pprint import pprint
from math import log
import numpy as np
import sys
from typing import List, Tuple, Callable, Iterable
from functools import lru_cache, reduce, cmp_to_key
from itertools import (
    starmap,
    accumulate,
    chain,
    islice,
    product,
    tee,
    pairwise,
    combinations,
)
from collections import Counter, deque
from heapq import heapify, heappushpop, heappop, heappush
from operator import add, neg, itemgetter
from bisect import bisect_right
import matplotlib.pyplot as plt

import arrays
import trees
import lists
import graphs
import search
import strings
import matrix
import mathematics
from importlib import reload

for m in (arrays, trees, lists, graphs, search, strings, matrix, mathematics):
    reload(m)

import arrays as ar
import trees as tr
import lists as ll
import graphs as g
import strings as s
import matrix as mt
import mathematics as m

print()

In [429]:
m._prime_table = []

In [446]:
len(list(m.prime_numbers(100000)))

9592

In [447]:
len(m._prime_table)

49999

In [448]:
list(m.prime_numbers(42))

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41]

In [381]:
list(m.prime_numbers(82))

[2,
 3,
 5,
 7,
 11,
 13,
 17,
 19,
 23,
 29,
 31,
 37,
 41,
 43,
 47,
 53,
 59,
 61,
 67,
 71,
 73,
 79]

In [436]:
m.prime_sum(4)

(2, 2)

In [434]:
m.prime_sum(9)

(2, 7)

In [433]:
m.prime_sum(6)

(3, 3)

In [449]:
mx = (0, 0)
for i in reversed(range(34096, 10000)):
    mx = max(mx, m.prime_sum(i))
print(mx)

KeyboardInterrupt: 

In [227]:
list(range(3, 6 - 2, 2))

[3]

In [371]:
173 + 7253

7426

In [375]:
(173 + 33923)

34096